**Imports and Inits**

In [ ]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

sz=180
arch=resnext50
bs=24
GENERATED_OUTPUT_DIR = 'numta/Fastai_gen/'

**Load 3 different model for voting** (actually we used TTA() with max_zoom=1.1 during submission. As it can generate random result, here only prediction is used)

In [ ]:
#tfms = tfms_from_model(arch, sz, max_zoom=1.1)
tfms = tfms_from_model(arch, sz)
data = ImageClassifierData.from_paths(GENERATED_OUTPUT_DIR, test_name='test', tfms=tfms, bs=bs, num_workers=4)

In [ ]:
learn = ConvLearner.pretrained(arch, data, ps=0.5)
learn.load('180_all_50_lrg')

In [ ]:
learn2 = ConvLearner.pretrained(arch, data, ps=0.5)
learn2.load('180_all_50_lrg3')

In [ ]:
learn3 = ConvLearner.pretrained(arch, data, ps=0.5)
learn3.load('180_all_50_lrg4')

**Prediction**

In [ ]:
test_probs, y = learn.TTA(is_test=True)
test_probs = np.mean(test_probs, axis=0)

In [ ]:
test_probs2, y2 = learn2.TTA(is_test=True)
test_probs2 = np.mean(test_probs2, axis=0)

In [ ]:
test_probs3, y3 = learn3.TTA(is_test=True)
test_probs3 = np.mean(test_probs3, axis=0)

In [ ]:
test_probs_comb = np.mean(np.array([test_probs, test_probs2, test_probs3]), axis=0)

In [ ]:
test_preds = np.argmax(test_probs_comb, axis=1)
file_names = np.array([os.path.basename(data.test_ds.fnames[x]) for x in range(len(test_preds))])
csv_data = []
for i, filename in enumerate(file_names):
    csv_data.append([filename, test_preds[i]])
csv_data = sorted(csv_data, key=lambda x: x[0])

columns = ['key', 'label']

df = pd.DataFrame(csv_data, columns=columns)

df.to_csv('submit.csv', index=False)
print (df.head())